Prediction/Initial_54/ 폴더에 있는 이미지 파일명을 하나씩 읽어

같은 이름의 원본 파일을 tree/train/에서 로드해

process_and_draw() 후처리 (tree 하나, 주변 branch/gnarl/crown 필터 등) 를 거친 뒤

Prediction/Initial_54_processed/ 폴더에 같은 파일명으로 저장합니다.
원본이나 기존 Prediction 폴더는 전혀 건드리지 않고, 새 폴더에만 처리 결과가 쌓입니다.

In [ ]:
import os
import cv2
from ultralytics import YOLO
from processing import filter_predictions  # 후처리 필터 함수
from processing import CLASS_COLORS         # 색상 매핑 딕셔너리
from processing import D_THRESH             # 반경 기준

# ──────────────────────────────────────
# 설정
# ──────────────────────────────────────
MODEL_PATH    = "runs/train/Initial_54/weights/best.pt"
SRC_ORIG_DIR  = "tree/train"
SRC_PRED_DIR  = "Prediction/Initial_54"
OUT_DIR       = "Prediction/Initial_54_processed"

# 모델 로드 & 클래스 이름
model        = YOLO(MODEL_PATH)
class_names  = model.names

# 출력 폴더 생성
os.makedirs(OUT_DIR, exist_ok=True)

# ──────────────────────────────────────
# 이미지 처리 함수
# ──────────────────────────────────────
def process_image(fn):
    orig_path = os.path.join(SRC_ORIG_DIR, fn)
    img = cv2.imread(orig_path)
    if img is None:
        print(f"⚠️ 원본 누락: {fn}")
        return

    res   = model.predict(source=orig_path, verbose=False)[0]
    boxes = res.boxes.xyxy.cpu().numpy()   # (N,4)
    confs = res.boxes.conf.cpu().numpy()   # (N,)
    clses = res.boxes.cls.cpu().numpy()    # (N,)

    # 후처리 필터링
    keep_idxs = filter_predictions(boxes, confs, clses, class_names)

    # 박스 그리기
    out = img.copy()
    for i in keep_idxs:
        x1, y1, x2, y2 = map(int, boxes[i])
        cls_id = int(clses[i])
        nm     = class_names[cls_id]
        conf   = confs[i]
        color  = CLASS_COLORS.get(nm, (0,255,0))

        cv2.rectangle(out, (x1, y1), (x2, y2), color, 2)
        txt = f"{nm}:{conf:.2f}"
        (tw, th), _ = cv2.getTextSize(txt, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        ty = max(y1 - 5, th + 5)
        cv2.rectangle(out, (x1, ty-th-4), (x1+tw+4, ty), color, -1)
        cv2.putText(out, txt, (x1+2, ty-2),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                    (255,255,255), 2, cv2.LINE_AA)

    out_path = os.path.join(OUT_DIR, fn)
    cv2.imwrite(out_path, out)
    print(f"✅ Saved: {out_path}")

# ──────────────────────────────────────
# 배치 처리
# ──────────────────────────────────────
files = [f for f in os.listdir(SRC_PRED_DIR) if f.lower().endswith((".jpg", ".png"))]
for fn in files:
    process_image(fn)

print("모든 후처리 완료!")


✅ Saved: Prediction/Initial_54_processed\나무_10_남_00969.jpg
✅ Saved: Prediction/Initial_54_processed\나무_10_남_01824.jpg
✅ Saved: Prediction/Initial_54_processed\나무_11_남_02046.jpg
✅ Saved: Prediction/Initial_54_processed\나무_11_여_04437.jpg
✅ Saved: Prediction/Initial_54_processed\나무_12_여_10387.jpg
✅ Saved: Prediction/Initial_54_processed\나무_13_남_07481.jpg
✅ Saved: Prediction/Initial_54_processed\나무_8_남_02483.jpg
✅ Saved: Prediction/Initial_54_processed\나무_9_여_05113.jpg
모든 후처리 완료!


test/ 폴더 내의 모든 이미지(확장자 .jpg.png)를 순차적으로 불러와서

모델(best.pt)으로 예측 수행

다섯 개 클래스(branch, root, crown, fruit, gnarl) 각각에 대해 탐지된 바운딩박스가 하나라도 있으면 y, 없으면 n 표시

tree 클래스의 가장 큰 바운딩박스 하나를 골라

그 박스의 중심 좌표로 이미지 캔버스 내 위치(loc: left,center,right) 지정

그 박스 면적 대비 전체 캔버스 면적으로 small,middle,large 구분

결과를 pandas DataFrame으로 모아 CSV로 저장

In [ ]:
import os
import cv2
import math
import pandas as pd
from tqdm import tqdm
from ultralytics import YOLO
from utils import GetModelPredictResult

# ───────────────────────────────────────────────────
# 1) 환경 설정
# ───────────────────────────────────────────────────
MODEL_PATH     = "runs/train/Finetuned_428/weights/best.pt"
TEST_DIR       = "tree/test"    # 예측할 이미지 폴더
OUTPUT_CSV     = "tree_test.csv"  # 저장할 결과 파일명
IMG_EXTS       = (".jpg", ".png")
LOC_THRESHOLDS = {
    "left":   (0.0,  0.33),
    "center": (0.33, 0.66),
    "right":  (0.66, 1.0),
}
SIZE_THRESHOLDS = {
    "small":  (0.0,  0.1),   # 박스 면적/전체 면적
    "middle": (0.1, 0.3),
    "big":  (0.3,  1.0),
}
CLASS_NAMES = ["branch", "root", "crown", "fruit", "gnarl"]

# ───────────────────────────────────────────────────
# 2) 모델 로드
# ───────────────────────────────────────────────────
model = YOLO(MODEL_PATH)

# ───────────────────────────────────────────────────
# 3) 예측할 이미지 목록
# ───────────────────────────────────────────────────
imgs = sorted([f for f in os.listdir(TEST_DIR) if f.lower().endswith(IMG_EXTS)])

# ───────────────────────────────────────────────────
# 4) 결과 담을 리스트
# ───────────────────────────────────────────────────
rows = []

# ───────────────────────────────────────────────────
# 5) 예측 실행
# ───────────────────────────────────────────────────
for img_name in tqdm(imgs, desc="Processing test images"):
    img_path = os.path.join(TEST_DIR, img_name)
    img = cv2.imread(img_path)
    h, w = img.shape[:2]
    area_total = w * h

    # 5.1) 모델 예측
    res = GetModelPredictResult(model, img_path)
    xyxy  = res.boxes.xyxy.cpu().numpy()  # N×4 배열
    clses = res.boxes.cls.cpu().numpy()   # N 길이

    # 5.2) 클래스별 y/n 플래그
    flags = {c: "n" for c in CLASS_NAMES}
    for cls in clses:
        name = model.names[int(cls)]
        if name in flags:
            flags[name] = "y"

    # 5.3) tree 박스 위치/크기 계산
    tree_boxes = [
        (x1, y1, x2, y2)
        for (x1, y1, x2, y2), cls in zip(xyxy, clses)
        if model.names[int(cls)] == "tree"
    ]
    loc, size = "", ""
    if tree_boxes:
        # 면적으로 가장 큰 박스 선택
        areas = [((x2 - x1) * (y2 - y1), (x1, y1, x2, y2)) for (x1, y1, x2, y2) in tree_boxes]
        max_area, box = max(areas, key=lambda x: x[0])
        x1, y1, x2, y2 = box
        cx = ((x1 + x2) / 2) / w  # [0,1] 범위

        # loc 결정
        for key, (lo, hi) in LOC_THRESHOLDS.items():
            if lo <= cx < hi:
                loc = key
                break

        # size 결정
        ratio = max_area / area_total
        for key, (lo, hi) in SIZE_THRESHOLDS.items():
            if lo <= ratio < hi:
                size = key
                break

    # 5.4) 한 행으로 합치기
    row = {
        "id":        os.path.splitext(img_name)[0],
        "branch_yn": flags["branch"],
        "root_yn":   flags["root"],
        "crown_yn":  flags["crown"],
        "fruit_yn":  flags["fruit"],
        "gnarl_yn":  flags["gnarl"],
        "loc":       loc,
        "size":      size,
    }
    rows.append(row)

# ───────────────────────────────────────────────────
# 6) DataFrame → CSV 저장
# ───────────────────────────────────────────────────
df = pd.DataFrame(rows)

# encoding="utf-8-sig" 인자가 없으면 한글이 깨짐
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print(f"✅ 예측 결과를 {OUTPUT_CSV}에 저장했습니다.")


Processing test images: 100%|██████████| 2520/2520 [01:15<00:00, 33.50it/s]

✅ 예측 결과를 tree_test.csv에 저장했습니다.


tree/train 폴더에서 이미지 10,080장 중 랜덤으로 하나 선택

오른쪽 창에 processing.py의 filter_predictions 함수로 후처리(crown 위/아래 필터링 포함)한 결과 표시

키 입력

스페이스(space): 다음 이미지

Q: 종료

In [2]:
import os
import random
import cv2
import numpy as np
import shutil
from ultralytics import YOLO
from processing import filter_predictions
from utils import GetModelPredictResult

# 설정
MODEL_PATH  = "runs/train/Finetuned_428/weights/best.pt"
IMG_DIR     = "tree/train"
SAVE_DIR = "review/images"
MAX_W, MAX_H = 1600, 900  # 최대 창 크기

# 모델 로드
model = YOLO(MODEL_PATH)
img_paths = [os.path.join(IMG_DIR, f)
             for f in os.listdir(IMG_DIR)
             if f.lower().endswith(('.jpg', '.png'))]

def draw_boxes(img, boxes, classes, confs, names, color_map):
    out = img.copy()
    for (x1,y1,x2,y2), cls, conf in zip(boxes, classes, confs):
        x1,y1,x2,y2 = map(int, (x1,y1,x2,y2))
        label = f"{names[int(cls)]}:{conf:.2f}"
        color = color_map.get(names[int(cls)], (255,255,255))
        cv2.rectangle(out, (x1,y1), (x2,y2), color, 2)
        cv2.putText(out, label, (x1, y1-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    return out

CLASS_COLORS = {
    "tree":   (255,   0,   0),
    "branch": (  0, 255,   0),
    "root":   (  0,   0, 255),
    "crown":  (255,   0, 255),
    "fruit":  (  0, 255, 255),
    "gnarl":  (128,   0, 128),
}

print("Press SPACE for next image, Q to quit.")
while True:
    img_path = random.choice(img_paths)
    img = cv2.imread(img_path)
    if img is None:
        continue

    # 1) Raw prediction
    res = GetModelPredictResult(model, img_path)
    boxes = res.boxes.xyxy.cpu().numpy()
    classes = res.boxes.cls.cpu().numpy()
    confs = res.boxes.conf.cpu().numpy()
    names = model.names
    left = draw_boxes(img, boxes, classes, confs, names, CLASS_COLORS)

    # 2) Post-processed prediction
    keep = filter_predictions(boxes, confs, classes, names,
                              crown_mode="above")
    right = draw_boxes(img,
                       boxes[keep], classes[keep], confs[keep],
                       names, CLASS_COLORS)

    # 3) 양쪽 붙이기
    h = max(left.shape[0], right.shape[0])
    w1, w2 = left.shape[1], right.shape[1]
    canvas = np.zeros((h, w1+w2, 3), dtype=np.uint8)
    canvas[:left.shape[0], :w1] = left
    canvas[:right.shape[0], w1:] = right

    # 4) 최대 사이즈에 맞춰 스케일링
    scale = min(MAX_W/(w1+w2), MAX_H/h, 1.0)
    if scale < 1.0:
        new_size = (int((w1+w2)*scale), int(h*scale))
        canvas = cv2.resize(canvas, new_size)

    # 5) 화면에 띄우기
    cv2.imshow("Raw (Left) vs Post-Processed (Right)", canvas)
    key = cv2.waitKey(0) & 0xFF
    cv2.destroyAllWindows()
    if key in (ord('q'), ord('Q')):
        break
    elif key in (ord('s'), ord('S')):
        dst = os.path.join(SAVE_DIR, os.path.basename(img_path))
        shutil.copy2(img_path, dst)
        print(f"✔️ Saved original to {dst}")
    # space 누르면 다음 반복

cv2.destroyAllWindows()


Press SPACE for next image, Q to quit.


뽑은 tree_test.csv 의 tree 미탐 수 분석

In [15]:
import pandas as pd

df = pd.read_csv('tree_test.csv')
missing = df['loc'].isna().sum() + (df['loc'] == '').sum()
print(f"Total images: {len(df)}, Tree not detected: {missing}")


Total images: 2520, Tree not detected: 0
